In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from neuralforecast.core import NeuralForecast
from neuralforecast.models import NHITS, PatchTST, BiTCN

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/marcopeix/time-series-analysis/master/data/medium_views_published_holidays.csv')
df['ds'] = pd.to_datetime(df['ds'])

In [4]:
df['].value_counts()

unique_id  ds          y     published  is_holiday
0          2020-01-01  1131  0.0        1             1
           2022-07-07  2002  0.0        0             1
           2022-07-15  707   0.0        0             1
           2022-07-14  795   0.0        0             1
           2022-07-13  954   0.0        0             1
                                                     ..
           2021-04-04  998   0.0        0             1
           2021-04-03  958   0.0        0             1
           2021-04-02  1105  0.0        0             1
           2021-04-01  1261  0.0        0             1
           2023-10-12  2262  0.0        0             1
Name: count, Length: 1381, dtype: int64